In [20]:
# libraries

from datetime import datetime
import os
import glob
import requests 
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup

# Web scrapping

In [ ]:
# download data 
# =============

link = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vSc_2y5N0I67wDU38DjDh35IZSIS30rQf7_NYZhtYYGU1jJYT6_kDx4YpF-qw0LSlGsBYP8pqM_a1Pd/pubhtml#'
req = requests.get(link)
soup = BeautifulSoup(req.content, "html.parser")

In [ ]:
tbody = soup.find_all('tbody')[0]
body = tbody.find_all('tr')

# print(tbody)
# print(body)

In [ ]:
head_row = [i.text for i in body[0].find_all('td')]
head_row

In [ ]:
contents = []

for i in range(len(body)):
    contents.append([i.text for i in body[i].find_all('td')])

# Saving to Dataframe

In [ ]:
p_df = pd.DataFrame(contents[2:len(contents)], columns=head_row)
p_df.head()

# Data cleaning and transformations

In [ ]:
# shape of dataframe
# ==================

p_df.shape

In [ ]:
# columns
# =======

p_df.columns

In [ ]:
# selecting important columns only
# ================================

p_df = p_df.loc[:, :'Backup Notes']

In [ ]:
# looking for missing values 
# ==========================

p_df.isna().sum()

In [ ]:
# no. of empty strings in each column
# ===================================

print(p_df.shape)

for i in p_df.columns:
    print(i, '\t', p_df[p_df[i]==''].shape[0])

In [ ]:
# replacing empty strings with np.nan
# ==================================-

print(p_df.shape)

p_df = p_df.replace(r'', np.nan, regex=True)
p_df.isna().sum()

In [ ]:
# droping empty rows (row with just row number but without patient entry
# ======================================================================

p_df.dropna(subset=['Detected State'], inplace=True)
print(p_df.shape)
p_df.isna().sum()

In [ ]:
p_df.columns

In [ ]:
# rename dateframe columns 
# ========================

p_df.columns = ['_'.join(col.lower().split()) for col in p_df.columns]
p_df.rename(columns = {'contracted_from_which_patient_(suspected)':'suspected_contacted_patient'}, inplace=True)
p_df.sample(5)

In [ ]:
# creating patient id column from patient number
# ===============================================

p_df['p_id'] = p_df['patient_number'].apply(lambda x : 'P'+str(x))
p_df.columns

In [ ]:
p_df = p_df.loc[:, :'backup_notes']

# Saving Data

In [ ]:
p_df.sample(5)

In [ ]:
p_df.to_csv('patients_data.csv', index=False)